In [1]:
import sys
import time
import datetime
import os
import tensorflow as tf
import argparse
import json
import socket
from PIL import Image
from util import *
import numpy as np
%load_ext autoreload
%autoreload 2
from copy import deepcopy
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from matplotlib.colors import hsv_to_rgb
%matplotlib inline

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
logPath = "training/"
snapshotPath = "snapshots/"

In [ ]:
with open("hyperParams.json") as f:
    instanceParams = json.load(f)

instanceParams

In [ ]:
printFrequency = instanceParams["printFreq"]
batchSize = instanceParams["batchSize"]

iterations = instanceParams["iterations"]
baseLearningRate = instanceParams["baseLR"]
learningRate = baseLearningRate
snapshotFrequency = instanceParams["snapshotFreq"]

from dotmap import DotMap
arg = DotMap()
arg.logDev = False
arg.resume = 'y'
resume, startIteration, snapshotFiles = checkResume(snapshotPath,logPath, arg)
iterations = 1000 * 20

### Training

In [ ]:
with tf.device("/gpu:0"):
    trainingData = TrainingData(batchSize,instanceParams)

with tf.device("/gpu:0"):
    # init
    with tf.variable_scope("netShare"):
        networkBodyF = NetworkBody(trainingData,instanceParams)
    with tf.variable_scope("netShare",reuse=True):
        networkBodyB = NetworkBody(trainingData,instanceParams,flipInput=True)

    trainingLoss = TrainingLoss(instanceParams,networkBodyF,networkBodyB,trainingData)
    solver,learningRateTensor = attachSolver(trainingLoss.loss)

    # loss scheduling
    recLossBWeightTensor = trainingLoss.recLossBWeight

In [ ]:
# merge summaries
merged = tf.summary.merge_all()
# saver
saver = tf.train.Saver(max_to_keep=0)

In [ ]:
printFrequency = 36
snapshotFrequency = 2000
iterations = 10000
print(printFrequency)
print(snapshotFrequency)
print(iterations)
startIteration = 500000
print(startIteration)

In [ ]:
# start
with sessionSetup(arg) as sess:
#     if resume:
#         saver.restore(sess,snapshotPath+snapshotFiles[-1][:-6])
#     else:
#         sess.run(tf.initialize_all_variables())
    saver.restore(sess,
                  '../model_download_scripts/photometric_smoothness/weights/iter_0000000000500000.ckpt')

    trainingData.dataQueuer.start_queueing(sess)

    #start summary writer
    summary_writer = tf.summary.FileWriter(logPath, sess.graph)

    #run
    lastPrint = time.time()
    for i in range(startIteration, startIteration + iterations + 10):
        # scheduled values
        learningRate = learningRateSchedule(baseLearningRate, i)
        recLossBWeight = unsupLossBSchedule(i)

         #run training
        feed_dict = {
            learningRateTensor: learningRate,
            recLossBWeightTensor: recLossBWeight,
        }
        summary,result,totalLoss = sess.run([merged,solver,trainingLoss.loss], feed_dict=feed_dict)

        if (i+1) % printFrequency == 0:
            timeDiff = time.time() - lastPrint
            itPerSec = printFrequency/timeDiff
            remainingIt = startIteration + iterations + 10 - i
            eta = remainingIt/itPerSec
            print("Iteration "+str(i+1)+": loss: "+str(totalLoss)+", iterations per second: "+str(itPerSec)+", ETA: "+str(datetime.timedelta(seconds=eta)))+", lr: "+str(learningRate)

            summary_writer.add_summary(summary,i+1)
            summary_writer.flush()
            lastPrint = time.time()

        if (i+1) % snapshotFrequency == 0:
            saver.save(sess,"snapshots/iter_"+str(i+1).zfill(16)+".ckpt")

        sys.stdout.flush()

    #close queing
    trainingData.dataQueuer.close(sess)

### Testing

In [ ]:
datasetRoot = '../example_data/'
frame0Path = '../example_data/datalists/hockey_im0.txt'
frame1Path = '../example_data/datalists/hockey_im1.txt'
gt0Path = '../example_data/datalists/hockey_gt0.txt'
desiredHeight = 480
desiredWidth = 854

In [ ]:
with open(frame0Path) as f:
    imagePairs0 = [datasetRoot+x[:-1] for x in f.readlines()]
with open(frame1Path) as f:
    imagePairs1 = [datasetRoot+x[:-1] for x in f.readlines()]
with open(gt0Path) as f:
    gtPairs1 = [datasetRoot+x[:-1] for x in f.readlines()]

In [ ]:
testData = TestData(imagePairs0,imagePairs1,gtPairs1,1,desiredHeight,desiredWidth)

In [ ]:
with tf.device("/gpu:0"):
    with tf.variable_scope("netShare"):
        networkBody = NetworkBody(testData,instanceParams)
    flowFinal = networkBody.flows[0]
    gt = testData.frame0["gt"]
    sLoss = asymmetricSmoothLoss(flowFinal, gt, instanceParams, 1, 1, None, instanceParams["boundaryAlpha"], True)
    sGrad = tf.gradients(sLoss, flowFinal)[0]

In [ ]:
flowViz = flowToRgb(flowFinal)
transformGrid = flowTransformGrid(flowFinal)
mean = tf.expand_dims(tf.expand_dims(tf.expand_dims([0.407871, 0.457525, 0.481094], 0), 0), 0)

In [ ]:
saver = tf.train.Saver()

# config tensorflow
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
iterations = 510000

In [ ]:
test_length = 70
start = 0

flows = []
viz = []
gradients = []
ground_truths = []
with tf.Session(config=config) as sess:
    saver.restore(sess,"snapshots/iter_"+str(iterations).zfill(16)+".ckpt")

    # run
    lastPrint = time.time()
    for i in range(start, start + test_length):
        feed_dict = {
            testData.im0File: imagePairs0[i],
            testData.im1File: imagePairs1[i],
            testData.gt0File: gtPairs1[i]
        }
        hsv, f, g, t = sess.run([flowViz, flowFinal, sGrad, gt],feed_dict=feed_dict)

        gradients.append(deepcopy(g))
        ground_truths.append(deepcopy(t))
        flows.append(deepcopy(f))
        h, w = 480, 854
        arr = np.maximum(np.minimum(np.asarray(hsv),1), 0)
        arr = np.squeeze(np.asarray(arr*255,np.uint8))
        im = Image.fromarray(arr[:h,:w,:])
        viz.append(deepcopy(im))
        # im.save("{}/{}.png".format(result_dir, str(i).zfill(3)))

In [ ]:
def plot_smooth_grad(i, overlay=True, sparsity=32):
    # optical flow hsv
    plt.figure(frameon=False, figsize=(15,10))
    plt.imshow(viz[i])

    # segmentation mask
    if overlay:
        plt.imshow(np.reshape(ground_truths[i][0], (480, 854)), cmap='gray', alpha=0.1)

    # optical flow vector field
    X, Y = np.mgrid[0:854, 0:480]
    est = flows[i][0]
    est = np.transpose(est, (1, 0, 2))
    U, V = est[:,:,0], est[:,:,1]
    plt.quiver(X[::sparsity, ::sparsity],
               Y[::sparsity, ::sparsity],
               U[::sparsity, ::sparsity],
               V[::sparsity, ::sparsity],
               color='black',
               edgecolor='k', alpha=0.8)

    # gradient vector field
    sgrad = np.transpose(gradients[0][0], (1, 0, 2))
    U, V = sgrad[:,:,0], sgrad[:,:,1]
    plt.quiver(X[::sparsity, ::sparsity],
               Y[::sparsity, ::sparsity],
               U[::sparsity, ::sparsity],
               V[::sparsity, ::sparsity],
               color='yellow',
               edgecolor='k', alpha=0.8)
    plt.axis('off')

In [ ]:
plot_smooth_grad(8, True)